In [ ]:
import os

os.getcwd()

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from pytorch_efficientnet import EfficientNetwork
from new_torch import train_model
from new_train import dataloaders, dataset_sizes



In [ ]:
batch_size = 20
num_workers = 0
model = EfficientNetwork(output_size=2, b1=False, b2=True)
learning_rate = 0.0001
weight_decay = 0.1
epochs = 10

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = EfficientNetwork(output_size=2, b1=False, b2=True)
# Criterion
criterion = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max',
                                                       patience=1, factor=0.4)

In [ ]:
dics = {}
dics['dataloaders'] = dataloaders
dics['dataset_sizes'] = dataset_sizes

In [ ]:
dataloaders

In [ ]:
mdl = train_model(model.to(device), criterion, optimizer, scheduler, dics, num_epochs=25)

In [ ]:
test_loader = dataloaders['test']

all_preds = torch.empty(0, 2)
all_labels = []
for imgs, label in test_loader:
    imgs_pred = mdl(imgs)

    all_preds = torch.cat((all_preds, imgs_pred))
    all_labels += label

In [ ]:
labels = [i.item() for i in all_labels]

In [ ]:
import pandas as pd

preds_prob = torch.softmax(all_preds, dim=1)
preds_prob = pd.DataFrame(preds_prob.detach().numpy())
preds_prob.columns = ['False', 'True']
preds_prob['Label'] = labels

In [ ]:
preds_prob.sort_values('True', ascending=False)

In [ ]:
from sklearn.metrics import roc_auc_score

preds_prob[preds_prob['Label'] == 1].sort_values('True', ascending=False)
roc_auc_score(preds_prob['Label'], preds_prob['True'])

In [ ]:
import numpy as np

print(np.mean(preds_prob.loc[preds_prob['True'] >= 0.9, 'Label']))
print(np.mean(preds_prob.loc[preds_prob['True'] < 0.9, 'Label']))

In [ ]:
torch.save(mdl, 'model.pth')